# High dimensional setting

The purpose of this notebook it is check the computation time for the simulation study in the high dimensional setting

First we load the libraries

In [1]:
# scripts and libraries
remove(list = ls())
options(warn = -1)
library(data.table)
library(magrittr)
library(ggplot2)
library(latex2exp)
library(xtable)

source("R_scripts/simulations.R")
source("R_scripts/vectorial_methods.R")
source("R_scripts/auxiliar_methods.R")

Simulation parameters

In [2]:
Tt <- 100 # series length
m <- 300 # series' dimension
r <- 250 # stable space's dimension
case <- 1 # scenario to consider
S <- 500 # number of simulations
crit <- "SC(n)" # criterio Johansen estimation
ec_det <- "none" # non trend and seasonality for Johansen procedure
pct <- "5pct" # Johansen's significance level
test <- "kpss" # test to use in PLS o PCA options: "adf" and "kpss"
persistence <- "low" ; dist <- "t" # persistence and innovation process distribution
trend <- FALSE # consider a simple linear trend
dependence <- TRUE # if we consider a covariance structure in the innovation process
seeds <- c(1,1) # seeds for reproducibility
sigma <- 1 # equal variance for the non-correlated case
methods_colors <- c("Johansen" = "#fa9f99","PLS" = "#4dcf74", "PCA" = "#90baff") 

Parameters definition

In [3]:
# Shared Sigma_eps
set.seed(seeds[1])  # for reproducibility
if (dependence) {
  Sigma_eps <- clusterGeneration::genPositiveDefMat(m, covMethod = "eigen")$Sigma
} else {
  Sigma_eps <- sigma * diag(1, m, m)
}

# Shared stable basis
set.seed(seeds[2]+r)  # shared seed for beta, beta_orto, alpha, gamma
I_m <- qr.Q(qr(matrix(rnorm(m^2), m, m)))
beta <- I_m[, 1:r]
beta_orto <- if (r < m) I_m[, (r + 1):m] else matrix(0, m, m)

# Alpha and gamma
alpha <- if (persistence == "low") runif(r, 0.1, 0.3) else runif(r, 0.3, 0.7)
gamma <- runif(ncol(beta_orto), -0.7, 0.7)

# Integration orders by case
orders <- switch(
  as.character(case),
  "1" = c(m, 0),
  "2" = c(m - 3, 2),
  stop("Unsupported case")
)
i1 <- orders[1]
i2 <- orders[2]

Time series simulation

In [4]:
set.seed(1)
epsilon <- if (dist == "normal") {
    if (trend) {
      portes::varima.sim(n = Tt + 100, k = m, sigma = Sigma_eps, trend = rep(0.05, m), demean = rep(0, m))
    } else {
      portes::varima.sim(n = Tt + 100, k = m, sigma = Sigma_eps, demean = rep(0, m))
    }
  } else if (dist == "t") {
    if (trend) {
      portes::varima.sim(n = Tt + 100, k = m, sigma = Sigma_eps, trend = rep(0.05, m), innov.dist = "t", dft = 3, demean = rep(0, m))
    } else {
      portes::varima.sim(n = Tt + 100, k = m, sigma = Sigma_eps, innov.dist = "t", dft = 3, demean = rep(0, m))
    }
  }

# Simulate X using the improved core
X_sim <- X_simulation(seed = 1,
m = m, r = r, Tt = Tt + 100,
beta = beta, beta_orto = beta_orto,
alpha = alpha, gamma = gamma, Sigma_eps = Sigma_eps,
mix = TRUE, dist = dist, trend = trend)(i1,i2)

beta_teo <- X_sim$beta
X <- ts(scale(X_sim$X[101:nrow(X_sim$X), ]))  # remove burn-in
XX <- X[1:(nrow(X) - 1), ]
Y <- X[2:nrow(X), ]

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 



Now, we apply the PLS and PCA method